In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.ops.rnn_cell import LSTMCell, LSTMStateTuple
import helpers
import os
#LSTMStateTuple(c,h)에서 c는 hidden state h는 output

#tf.reset_default_graph()

In [2]:
lstm_size_encoder = 128
lstm_size_decoder = 256
vocab_size = 679 #띄어쓰기도 하나의 word야?
embed_size = 100# Document Vector로 Input된 총 문장수고 각 Vocab이 몇번 나왔는지 표기(?)
#max_length = 5 


#batch_size = 32
encoder_cell = LSTMCell(lstm_size_encoder)
decoder_cell = LSTMCell(lstm_size_decoder)
PAD = 0
EOS = 1

In [3]:
data_dir = "en_fr"
input_file_en = os.path.join(data_dir, "A_62_952_ADD1_en.snt")
lines_en = []
with open(input_file_en, "r", encoding="utf-8") as f_en:
    for line in f_en.readlines():
        line = line.replace('\n','')
        if not line == '':
            lines_en.append(line)
        
print(lines_en[-2])
#print(("Text loaded from '%s'") % (input_file))

A revised edition of the rules of procedure has been issued under the symbol A/520/Rev.17.


In [4]:
data_dir = "en_fr"
#save_dir = "eng_fre_dataset/multiUN.en/un/text/en-fr/2009"
input_file_fr = os.path.join(data_dir, "A_62_952_ADD1_fr.snt")
lines_fr = []
with open(input_file_fr, "r", encoding="utf-8") as f_fr:
    for line in f_fr.readlines():
        line = line.replace('\n','')
        if not line == '':
            lines_fr.append(line)
        
print(lines_fr[-2])
#print(("Text loaded from '%s'") % (input_file))

Une version révisée du Règlement intérieur a été publiée sous la cote A/520/Rev.17.


In [5]:
len(lines_en), len(lines_fr)

(67, 67)

In [6]:
word_data = []
for i in range(len(lines_en)):
    word_data.append((lines_en[i],lines_fr[i]))

In [7]:
x = [x for x, y in word_data]
y = [y for x, y in word_data]

In [8]:
x_list = []
for i in range(len(x)):
    x_text = x[i].split()
    x_list.append(x_text)
for i in range(1, len(x_list)):
    x_list[i] = set(x_list[i-1]).union(set(x_list[i]))
    x_words_set = x_list[len(x_list)-1]

In [9]:
y_list = []
for i in range(len(y)):
    y_text = y[i].split()
    y_list.append(y_text)
for i in range(1, len(y_list)):
    y_list[i] = set(y_list[i-1]).union(set(y_list[i]))
    y_words_set = y_list[len(y_list)-1]

In [10]:
word2numX = dict(zip(x_words_set, range(len(x_words_set))))
word2numY = dict(zip(y_words_set, range(len(y_words_set))))

In [11]:
num2wordX = dict(zip(word2numX.values(), word2numX.keys()))

In [12]:
# max_len 뽑음
numWordsList = []
for word in x:
    Words = word.split(' ')
    num = len(Words)
    numWordsList.append(num) 
max_len = max(numWordsList)

In [13]:
step1_X = [
    word.split() for word in x
]

In [14]:
second_list = []
for i in range(len(step1_X)):
    first_list = []
    for j in range(len(step1_X[i])):
        k = word2numX[step1_X[i][j]]
        first_list.append(k)
        padded_list = first_list
    second_list.append(padded_list)
x = second_list
x = np.array(x)

In [15]:
num2wordY = dict(zip(word2numY.values(), word2numY.keys()))

In [16]:
step1_Y = [
    word.split() for word in y
]

In [17]:
second_list = []
for i in range(len(step1_Y)):
    first_list = []
    for j in range(len(step1_Y[i])):
        k = word2numY[step1_Y[i][j]]
        first_list.append(k)
        padded_list = first_list
    second_list.append(padded_list)
y = second_list
y = np.array(y)

In [18]:
print(len(word2numX))
print(len(word2numY))


679
720


In [19]:
encoder_inputs = tf.placeholder(tf.int32, (None, None), "encoder_inputs")
encoder_inputs_length = tf.placeholder(tf.int32, (None, ), "encoder_inputs_length")
decoder_targets = tf.placeholder(tf.int32, (None, None), "decoder_targets")

In [20]:
input_embedding = tf.Variable(tf.random_uniform((vocab_size, embed_size), -1.0, 1.0), name="embedding")
embedded_input = tf.nn.embedding_lookup(input_embedding, encoder_inputs)

In [21]:
((enc_fw_outputs, enc_bw_outputs),
(enc_fw_last_state, enc_bw_last_state)) = (tf.nn.bidirectional_dynamic_rnn
                                            (cell_fw=encoder_cell,
                                             cell_bw=encoder_cell,
                                             inputs=embedded_input,
                                             sequence_length=encoder_inputs_length,
                                             dtype=tf.float32, time_major=True))

In [22]:
encoder_outputs = tf.concat((enc_fw_outputs, enc_bw_outputs), 2)
enc_last_state_c = tf.concat((enc_fw_last_state.c, enc_bw_last_state.c),1)
enc_last_state_h = tf.concat((enc_fw_last_state.h, enc_bw_last_state.h),1)
encoder_last_state = LSTMStateTuple(
    c=enc_last_state_c,
    h=enc_last_state_h
)

In [23]:
decoder_cell = LSTMCell(lstm_size_decoder)

In [24]:
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))

In [25]:
decoder_lengths = encoder_inputs_length + 22

In [26]:
#manually specifying since we are going to implement attention details for the decoder in a sec
#weights
W = tf.Variable(tf.random_uniform([lstm_size_decoder, vocab_size], -1, 1), dtype=tf.float32)
#bias
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [27]:
assert EOS == 1 and PAD == 0

eos_time_slice = tf.ones([batch_size], dtype=tf.int32, name='EOS')
pad_time_slice = tf.zeros([batch_size], dtype=tf.int32, name='PAD')

#retrieves rows of the params tensor. The behavior is similar to using indexing with arrays in numpy
eos_step_embedded = tf.nn.embedding_lookup(input_embedding, eos_time_slice)
pad_step_embedded = tf.nn.embedding_lookup(input_embedding, pad_time_slice)

In [28]:
def loop_fn_initial():
    initial_elements_finished = (0 >= decoder_lengths)  # all False at the initial step
    #end of sentence
    initial_input = eos_step_embedded
    #last time steps cell state
    initial_cell_state = encoder_last_state
    #none
    initial_cell_output = None
    #none
    initial_loop_state = None  # we don't need to pass any additional information
    return (initial_elements_finished,
            initial_input,
            initial_cell_state,
            initial_cell_output,
            initial_loop_state)

In [29]:
#attention mechanism --choose which previously generated token to pass as input in the next timestep
def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):

    
    def get_next_input():
        #dot product between previous ouput and weights, then + biases
        output_logits = tf.add(tf.matmul(previous_output, W), b)
        #Logits simply means that the function operates on the unscaled output of 
        #earlier layers and that the relative scale to understand the units is linear. 
        #It means, in particular, the sum of the inputs may not equal 1, that the values are not probabilities 
        #(you might have an input of 5).
        #prediction value at current time step
        
        #Returns the index with the largest value across axes of a tensor.
        prediction = tf.argmax(output_logits, axis=1)
        #embed prediction for the next input
        next_input = tf.nn.embedding_lookup(input_embedding, prediction)
        return next_input
    
    
    elements_finished = (time >= decoder_lengths) # this operation produces boolean tensor of [batch_size]
                                                  # defining if corresponding sequence has ended

    
    
    #Computes the "logical and" of elements across dimensions of a tensor.
    finished = tf.reduce_all(elements_finished) # -> boolean scalar
    #Return either fn1() or fn2() based on the boolean predicate pred.
    input = tf.cond(finished, lambda: pad_step_embedded, get_next_input)
    
    #set previous to current
    state = previous_state
    output = previous_output
    loop_state = None

    return (elements_finished, 
            input,
            state,
            output,
            loop_state)

In [30]:
def loop_fn(time, previous_output, previous_state, previous_loop_state):
    if previous_state is None:    # time == 0
        assert previous_output is None and previous_state is None
        return loop_fn_initial()
    else:
        return loop_fn_transition(time, previous_output, previous_state, previous_loop_state)

#Creates an RNN specified by RNNCell cell and loop function loop_fn.
#This function is a more primitive version of dynamic_rnn that provides more direct access to the 
#inputs each iteration. It also provides more control over when to start and finish reading the sequence, 
#and what to emit for the output.
#ta = tensor array
decoder_outputs_ta, decoder_last_state, _ = tf.nn.raw_rnn(decoder_cell, loop_fn)
decoder_outputs = decoder_outputs_ta.stack()

In [31]:
decoder_outputs

<tf.Tensor 'TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 256) dtype=float32>

In [32]:
#to convert output to human readable prediction
#we will reshape output tensor

#Unpacks the given dimension of a rank-R tensor into rank-(R-1) tensors.
#reduces dimensionality
decoder_max_steps, decoder_batch_size, decoder_dim = tf.unstack(tf.shape(decoder_outputs))
#flettened output tensor
decoder_outputs_flat = tf.reshape(decoder_outputs, (-1, decoder_dim))
#pass flattened tensor through decoder
decoder_logits_flat = tf.add(tf.matmul(decoder_outputs_flat, W), b)
#prediction vals
decoder_logits = tf.reshape(decoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [33]:
#final prediction
decoder_prediction = tf.argmax(decoder_logits, 2)

In [34]:
#cross entropy loss
#one hot encode the target values so we don't rank just differentiate
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
    logits=decoder_logits,
)

#loss function
loss = tf.reduce_mean(stepwise_cross_entropy)
#train it 
train_op = tf.train.AdamOptimizer().minimize(loss)

In [35]:
def next_feed():
    batch = x
    batch_y = y
    encoder_inputs_, encoder_input_lengths_ = helpers.batch(batch)
    decoder_targets_, _ = helpers.batch(
        [(sequence) + [EOS] + [PAD] for sequence in batch_y]
    )
    return {
        encoder_inputs: encoder_inputs_,
        encoder_inputs_length: encoder_input_lengths_,
        decoder_targets: decoder_targets_,
    }

In [36]:
loss_track = []

In [38]:
max_batches = 11
batches_in_epoch = 10

save_dir = 'data/'

sess = tf.Session()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())
ckpt = tf.train.get_checkpoint_state(save_dir)

load_name = os.path.join(save_dir, 'model.ckpt-500')
print(load_name)
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, load_name)
    for batch in range(max_batches):
        fd = next_feed()
        _, l = sess.run([train_op, loss], fd)
        loss_track.append(l)
    
        if batch == 0 or batch % batches_in_epoch == 0:
            print('batch {}'.format(batch))
            print('  minibatch loss: {}'.format(sess.run(loss, fd)))
            predict_ = sess.run(decoder_prediction, fd)
            for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
                print('  sample {}:'.format(i + 1))
                print('    input     > {}'.format(inp))
                print('    prediction     > {}'.format(pred))


                if i >= 10:
                    break
            print()

data/model.ckpt-500
INFO:tensorflow:Restoring parameters from data/model.ckpt-500
batch 0
  minibatch loss: 7.027068138122559
  sample 1:
    input     > [314 250   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0]
    prediction     > [  0   0   0   0   0   0   0   0   0   0   0   0 118   0 447 447 447 447
 447 447 447 532 447 447   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

batch 10
  minibatch loss: 5.1847944259643555
  sample 1:
    input     > [314 250   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0]
    prediction     > [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0